In [32]:
# ----------------------------- This cell be detect hand and will remove background from image.------------------
import argparse
import glob
import os
import cv2
import pathlib
from PIL import Image
from rembg.bg import remove
from yolo import YOLO
from pathlib import Path

dataset_folder = "data"
resultfolder = "abcd"

yolo = YOLO("models/cross-hands.cfg", "models/cross-hands.weights", ["hand"])
yolo.size = 640
yolo.confidence = 0.40

conf_sum = 0
detection_count = 0

data_dir = pathlib.Path('data/')                 
file_paths = list(data_dir.glob('*/*.jpg'))

# files
for file in file_paths:
    
    path = Path(file)
    path =str(path).split("\\")
    path = path[-2]+"\\"+path[-1]
    path = os.path.join(resultfolder,  path)
    
    if not os.path.exists(path):
        file = os.path.join(file)
        mat = cv2.imread(file)
        mat = cv2.resize(mat, (256,256))
        width, height, inference_time, results = yolo.inference(mat)

        print("%s in %s seconds: %s classes found!" %
            (os.path.basename(file), round(inference_time, 2), len(results)))

        output = []
        for detection in results:
            id, name, confidence, x, y, w, h = detection
            cx = x + (w / 2)
            cy = y + (h / 2)

            conf_sum += confidence
            detection_count += 1

            # draw a bounding box rectangle and label on the image
            color = (255, 0, 255)
            segment = mat[y:y+h, x:x+w]
            
            if not os.path.exists(os.path.dirname(path)):
                try:
                    os.makedirs(os.path.dirname(path))
                except: # Guard against race condition
                    print("folder error")
            try:
                segment = cv2.resize(segment, (256,256))
                img = cv2.cvtColor(segment, cv2.COLOR_BGR2RGB)
                im_pil = Image.fromarray(img)
                cutout = remove(im_pil)
                cutout.save(path, "PNG")
#                 segment = cv2.cvtColor(segment,cv2.COLOR_BGR2GRAY)
#                 cv2.imwrite(path, segment)
            except:
                pass

108.jpg in 2.42 seconds: 0 classes found!
117.jpg in 1.47 seconds: 0 classes found!
120.jpg in 1.63 seconds: 0 classes found!
121.jpg in 1.44 seconds: 0 classes found!
136.jpg in 1.51 seconds: 1 classes found!
144.jpg in 1.96 seconds: 1 classes found!
145.jpg in 1.77 seconds: 0 classes found!
146.jpg in 1.71 seconds: 0 classes found!
147.jpg in 1.7 seconds: 0 classes found!
148.jpg in 1.98 seconds: 0 classes found!
149.jpg in 2.26 seconds: 0 classes found!
150.jpg in 2.34 seconds: 0 classes found!
151.jpg in 2.06 seconds: 0 classes found!
152.jpg in 1.71 seconds: 0 classes found!
153.jpg in 2.22 seconds: 0 classes found!
154.jpg in 1.68 seconds: 0 classes found!
155.jpg in 1.78 seconds: 0 classes found!
156.jpg in 1.94 seconds: 0 classes found!
157.jpg in 1.67 seconds: 0 classes found!
158.jpg in 1.84 seconds: 0 classes found!
159.jpg in 1.88 seconds: 0 classes found!
160.jpg in 2.04 seconds: 0 classes found!
161.jpg in 1.92 seconds: 0 classes found!
162.jpg in 2.12 seconds: 0 classes 

529.jpg in 2.48 seconds: 0 classes found!
53.jpg in 1.97 seconds: 1 classes found!
530.jpg in 3.82 seconds: 1 classes found!
531.jpg in 2.74 seconds: 0 classes found!
532.jpg in 2.07 seconds: 0 classes found!
533.jpg in 2.59 seconds: 0 classes found!
534.jpg in 1.93 seconds: 1 classes found!
535.jpg in 1.96 seconds: 1 classes found!
536.jpg in 2.26 seconds: 1 classes found!
537.jpg in 2.25 seconds: 0 classes found!
538.jpg in 2.25 seconds: 1 classes found!
539.jpg in 2.37 seconds: 1 classes found!
54.jpg in 2.5 seconds: 0 classes found!
55.jpg in 1.73 seconds: 1 classes found!
56.jpg in 2.11 seconds: 1 classes found!
57.jpg in 2.39 seconds: 1 classes found!
58.jpg in 2.42 seconds: 0 classes found!
59.jpg in 2.37 seconds: 0 classes found!
6.jpg in 1.73 seconds: 1 classes found!
60.jpg in 2.09 seconds: 0 classes found!
61.jpg in 1.92 seconds: 0 classes found!
62.jpg in 2.06 seconds: 0 classes found!
63.jpg in 1.92 seconds: 0 classes found!
64.jpg in 1.86 seconds: 0 classes found!
65.jpg 

221.jpg in 4.51 seconds: 1 classes found!
222.jpg in 2.75 seconds: 1 classes found!
223.jpg in 2.63 seconds: 1 classes found!
224.jpg in 2.5 seconds: 2 classes found!
225.jpg in 2.28 seconds: 1 classes found!
226.jpg in 2.7 seconds: 1 classes found!
227.jpg in 2.67 seconds: 1 classes found!
228.jpg in 2.47 seconds: 1 classes found!
229.jpg in 2.3 seconds: 1 classes found!
23.jpg in 2.37 seconds: 1 classes found!
230.jpg in 2.31 seconds: 1 classes found!
231.jpg in 2.33 seconds: 1 classes found!
232.jpg in 2.38 seconds: 1 classes found!
233.jpg in 2.57 seconds: 1 classes found!
234.jpg in 3.67 seconds: 1 classes found!
235.jpg in 3.03 seconds: 1 classes found!
236.jpg in 3.02 seconds: 1 classes found!
237.jpg in 2.64 seconds: 1 classes found!
238.jpg in 3.27 seconds: 1 classes found!
239.jpg in 2.67 seconds: 1 classes found!
24.jpg in 3.91 seconds: 1 classes found!
240.jpg in 2.47 seconds: 1 classes found!
241.jpg in 3.45 seconds: 1 classes found!
242.jpg in 2.84 seconds: 1 classes foun

399.jpg in 2.4 seconds: 1 classes found!
4.jpg in 2.29 seconds: 0 classes found!
40.jpg in 2.08 seconds: 1 classes found!
400.jpg in 2.52 seconds: 1 classes found!
401.jpg in 2.47 seconds: 1 classes found!
402.jpg in 2.49 seconds: 1 classes found!
403.jpg in 2.52 seconds: 1 classes found!
404.jpg in 2.55 seconds: 1 classes found!
405.jpg in 2.56 seconds: 1 classes found!
406.jpg in 2.58 seconds: 1 classes found!
407.jpg in 2.56 seconds: 1 classes found!
408.jpg in 3.56 seconds: 1 classes found!
409.jpg in 2.48 seconds: 1 classes found!
41.jpg in 2.34 seconds: 1 classes found!
410.jpg in 2.9 seconds: 1 classes found!
411.jpg in 2.74 seconds: 1 classes found!
412.jpg in 2.09 seconds: 1 classes found!
413.jpg in 1.99 seconds: 1 classes found!
414.jpg in 2.03 seconds: 1 classes found!
415.jpg in 2.2 seconds: 1 classes found!
416.jpg in 2.22 seconds: 1 classes found!
417.jpg in 2.76 seconds: 1 classes found!
418.jpg in 3.18 seconds: 1 classes found!
419.jpg in 3.03 seconds: 1 classes found!

89.jpg in 2.03 seconds: 1 classes found!
9.jpg in 2.39 seconds: 1 classes found!
90.jpg in 2.44 seconds: 0 classes found!
91.jpg in 2.07 seconds: 1 classes found!
92.jpg in 2.39 seconds: 1 classes found!
93.jpg in 2.37 seconds: 0 classes found!
94.jpg in 2.04 seconds: 0 classes found!
95.jpg in 2.15 seconds: 1 classes found!
96.jpg in 2.39 seconds: 0 classes found!
97.jpg in 1.99 seconds: 1 classes found!
98.jpg in 2.33 seconds: 1 classes found!
99.jpg in 2.34 seconds: 1 classes found!
1.jpg in 2.35 seconds: 1 classes found!
10.jpg in 2.39 seconds: 0 classes found!
100.jpg in 2.01 seconds: 1 classes found!
101.jpg in 2.39 seconds: 1 classes found!
102.jpg in 2.54 seconds: 1 classes found!
103.jpg in 2.45 seconds: 1 classes found!
104.jpg in 2.4 seconds: 1 classes found!
105.jpg in 2.42 seconds: 1 classes found!
106.jpg in 2.38 seconds: 1 classes found!
107.jpg in 2.4 seconds: 1 classes found!
108.jpg in 2.43 seconds: 1 classes found!
109.jpg in 2.43 seconds: 1 classes found!
11.jpg in 

...

207.jpg in 1.69 seconds: 1 classes found!
208.jpg in 1.64 seconds: 0 classes found!
209.jpg in 1.61 seconds: 0 classes found!
21.jpg in 1.64 seconds: 1 classes found!
210.jpg in 1.75 seconds: 1 classes found!
211.jpg in 1.71 seconds: 1 classes found!
212.jpg in 1.68 seconds: 1 classes found!
213.jpg in 1.78 seconds: 0 classes found!
214.jpg in 1.67 seconds: 0 classes found!
215.jpg in 1.7 seconds: 1 classes found!
216.jpg in 1.62 seconds: 1 classes found!
217.jpg in 1.63 seconds: 0 classes found!
218.jpg in 1.72 seconds: 0 classes found!
219.jpg in 1.75 seconds: 0 classes found!
22.jpg in 1.61 seconds: 1 classes found!
220.jpg in 1.65 seconds: 0 classes found!
221.jpg in 1.68 seconds: 0 classes found!
222.jpg in 1.67 seconds: 0 classes found!
223.jpg in 1.59 seconds: 1 classes found!
224.jpg in 1.68 seconds: 0 classes found!
225.jpg in 1.66 seconds: 1 classes found!
226.jpg in 1.62 seconds: 1 classes found!
227.jpg in 1.69 seconds: 0 classes found!
228.jpg in 1.7 seconds: 0 classes fou

72.jpg in 1.63 seconds: 1 classes found!
73.jpg in 1.7 seconds: 1 classes found!
74.jpg in 1.7 seconds: 0 classes found!
75.jpg in 1.63 seconds: 0 classes found!
76.jpg in 1.65 seconds: 1 classes found!
77.jpg in 1.75 seconds: 0 classes found!
78.jpg in 1.77 seconds: 0 classes found!
79.jpg in 1.66 seconds: 1 classes found!
8.jpg in 1.65 seconds: 1 classes found!
80.jpg in 1.7 seconds: 1 classes found!
81.jpg in 1.67 seconds: 1 classes found!
82.jpg in 1.7 seconds: 0 classes found!
83.jpg in 1.72 seconds: 0 classes found!
84.jpg in 1.61 seconds: 0 classes found!
85.jpg in 1.71 seconds: 0 classes found!
86.jpg in 1.73 seconds: 0 classes found!
87.jpg in 1.67 seconds: 0 classes found!
88.jpg in 1.7 seconds: 0 classes found!
89.jpg in 1.63 seconds: 0 classes found!
9.jpg in 1.68 seconds: 1 classes found!
90.jpg in 1.76 seconds: 1 classes found!
91.jpg in 1.63 seconds: 1 classes found!
92.jpg in 1.66 seconds: 0 classes found!
93.jpg in 1.66 seconds: 1 classes found!
94.jpg in 1.65 seconds:

25.jpg in 1.64 seconds: 1 classes found!
250.jpg in 1.61 seconds: 1 classes found!
251.jpg in 1.73 seconds: 1 classes found!
252.jpg in 1.65 seconds: 1 classes found!
253.jpg in 1.65 seconds: 1 classes found!
254.jpg in 1.61 seconds: 1 classes found!
255.jpg in 1.71 seconds: 1 classes found!
256.jpg in 2.56 seconds: 1 classes found!
257.jpg in 1.73 seconds: 2 classes found!
258.jpg in 1.66 seconds: 1 classes found!
259.jpg in 1.69 seconds: 1 classes found!
26.jpg in 1.75 seconds: 1 classes found!
260.jpg in 1.63 seconds: 1 classes found!
261.jpg in 1.62 seconds: 1 classes found!
262.jpg in 1.66 seconds: 0 classes found!
263.jpg in 1.59 seconds: 1 classes found!
264.jpg in 1.64 seconds: 1 classes found!
265.jpg in 1.65 seconds: 1 classes found!
266.jpg in 1.74 seconds: 0 classes found!
267.jpg in 1.71 seconds: 1 classes found!
268.jpg in 1.58 seconds: 0 classes found!
269.jpg in 1.41 seconds: 1 classes found!
27.jpg in 1.58 seconds: 1 classes found!
270.jpg in 1.46 seconds: 0 classes fo

149.jpg in 2.22 seconds: 1 classes found!
15.jpg in 1.63 seconds: 1 classes found!
150.jpg in 1.66 seconds: 1 classes found!
151.jpg in 1.62 seconds: 1 classes found!
152.jpg in 1.7 seconds: 1 classes found!
153.jpg in 1.78 seconds: 2 classes found!
154.jpg in 1.69 seconds: 1 classes found!
155.jpg in 1.85 seconds: 1 classes found!
156.jpg in 1.67 seconds: 0 classes found!
157.jpg in 1.6 seconds: 0 classes found!
158.jpg in 1.59 seconds: 0 classes found!
159.jpg in 1.76 seconds: 0 classes found!
16.jpg in 1.59 seconds: 1 classes found!
160.jpg in 1.69 seconds: 0 classes found!
161.jpg in 1.6 seconds: 1 classes found!
162.jpg in 1.63 seconds: 0 classes found!
163.jpg in 1.6 seconds: 0 classes found!
164.jpg in 1.73 seconds: 0 classes found!
165.jpg in 1.84 seconds: 0 classes found!
166.jpg in 2.59 seconds: 0 classes found!
167.jpg in 1.74 seconds: 0 classes found!
168.jpg in 1.71 seconds: 0 classes found!
169.jpg in 1.6 seconds: 0 classes found!
17.jpg in 1.62 seconds: 0 classes found!


4.jpg in 1.62 seconds: 0 classes found!
40.jpg in 1.63 seconds: 1 classes found!
41.jpg in 1.61 seconds: 0 classes found!
42.jpg in 1.75 seconds: 1 classes found!
43.jpg in 1.61 seconds: 0 classes found!
44.jpg in 1.64 seconds: 0 classes found!
45.jpg in 1.59 seconds: 1 classes found!
46.jpg in 1.61 seconds: 1 classes found!
47.jpg in 1.68 seconds: 0 classes found!
48.jpg in 1.61 seconds: 0 classes found!
49.jpg in 1.81 seconds: 1 classes found!
5.jpg in 1.62 seconds: 1 classes found!
50.jpg in 1.61 seconds: 0 classes found!
51.jpg in 1.76 seconds: 0 classes found!
52.jpg in 1.59 seconds: 1 classes found!
53.jpg in 1.68 seconds: 1 classes found!
54.jpg in 1.77 seconds: 0 classes found!
55.jpg in 1.59 seconds: 0 classes found!
56.jpg in 1.66 seconds: 1 classes found!
57.jpg in 1.6 seconds: 1 classes found!
58.jpg in 1.61 seconds: 1 classes found!
59.jpg in 1.63 seconds: 0 classes found!
6.jpg in 1.6 seconds: 2 classes found!
60.jpg in 1.67 seconds: 1 classes found!
61.jpg in 1.62 second

In [ ]:
# --------------------------------- Use this code to detect skin only ----------------------------------
import argparse
import glob
import os
import cv2
import pathlib
from pathlib import Path
import skin_detector

dataset_folder = "result/signs"
resultfolder = "skin"

conf_sum = 0
detection_count = 0

data_dir = pathlib.Path(f'{dataset_folder}/')                 
file_paths = list(data_dir.glob('*/*.jpg'))

# files
for file in file_paths:
    
    path = Path(file)
    path = str(path).split("\\")
    path = path[-2]+"\\"+path[-1]
    path = os.path.join(resultfolder,  path)
    
    if not os.path.exists(path):
        file = os.path.join(file)
        print(path)
        img_col = cv2.imread(file, 1)
    
        segment = skin_detector.process(img_col)

        if not os.path.exists(os.path.dirname(path)):
            try:
                os.makedirs(os.path.dirname(path))
            except: # Guard against race condition
                print("folder error")
        try:
            segment = cv2.bitwise_and(img_col, img_col, mask=segment)
            cv2.imwrite(path, segment)
        except Exception as e:
            print(e)